In [ ]:
import subprocess
import os

# Check JAVA_HOME
print("JAVA_HOME:", os.environ.get('JAVA_HOME', 'Not set'))

if os.environ.get('JAVA_HOME'):
    result = subprocess.run([os.environ['JAVA_HOME'] + '/bin/java', '-version'], 
                          capture_output=True, text=True)
    print("\nProject JAVA_HOME Java version:")
    print(result.stderr)

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("local-pyspark").getOrCreate()

In [ ]:
import pandas as pd

# Read csv
taxi_df = pd.read_csv("./nyc-yellow-taxi-trip-records-january-2024/nyc_tlc_yellow_2024_01.csv")

print(taxi_df.dtypes)

In [ ]:
# Convert to parquet

taxi_df.to_parquet("./nyc-yellow-taxi-trip-records-january-2024/nyc_tlc_yellow_2024_01.parquet")
taxi_df.head(5)

In [ ]:
import pyarrow.parquet as pq

# Check generated statistics in parquet file
pf = pq.ParquetFile("./nyc-yellow-taxi-trip-records-january-2024/nyc_tlc_yellow_2024_01.parquet")
print(f"Row groups: {pf.num_row_groups}")
for i in range(pf.num_row_groups):
    rg = pf.metadata.row_group(i)
    print(type(rg))

    print(f"\n--- Row Group {i} ({rg.num_rows:,} rows) ---")
    
    for j in range(rg.num_columns):
        print(f"\n--- Column {j} statistics ---")
        chunk = rg.column(j)
        print(chunk.statistics.to_dict)

In [ ]:
# Read parquet file in Spark session
spark.read.parquet("./nyc-yellow-taxi-trip-records-january-2024/nyc_tlc_yellow_2024_01.parquet")